In [10]:
pip install opencv-python opencv-python-headless tensorflow pygame mediapipe kaggle


In [12]:
import os
from google.colab import files

# Upload the kaggle.json file
uploaded = files.upload()

# Create the Kaggle directory
os.makedirs('/root/.kaggle', exist_ok=True)

# Move the kaggle.json file to the correct location
!mv kaggle.json /root/.kaggle/

# Set permissions
!chmod 600 /root/.kaggle/kaggle.json


Saving kaggle.json to kaggle (1).json


In [13]:
!kaggle datasets download -d dheerajperumandla/drowsiness-dataset


Dataset URL: https://www.kaggle.com/datasets/dheerajperumandla/drowsiness-dataset
License(s): unknown
 63% 101M/161M [00:00<00:00, 1.05GB/s]
100% 161M/161M [00:00<00:00, 847MB/s] 


In [14]:
import zipfile

# Path to the downloaded zip file
zip_file_path = '/content/drowsiness-dataset.zip'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drowsiness_dataset')

# List extracted files
os.listdir('/content/drowsiness_dataset')


['train']

In [23]:
for root, dirs, files in os.walk('/content/drowsiness_dataset'):
    print(root, dirs[:5], len(files))


/content/drowsiness_dataset ['train'] 0
/content/drowsiness_dataset/train ['yawn', 'Closed', 'Open', 'no_yawn'] 0
/content/drowsiness_dataset/train/yawn [] 723
/content/drowsiness_dataset/train/Closed [] 726
/content/drowsiness_dataset/train/Open [] 726
/content/drowsiness_dataset/train/no_yawn [] 725


In [25]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import os

img_size = 64

# Initialize lists
eye_images, eye_labels = [], []
mouth_images, mouth_labels = [], []

train_path = '/content/drowsiness_dataset/train'

# Eye images
for label in ['Open','Closed']:
    path = os.path.join(train_path, label)
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_size,img_size))
        eye_images.append(img)
        eye_labels.append(0 if label=='Open' else 1)

# Mouth images
for label in ['yawn','no_yawn']:
    path = os.path.join(train_path, label)
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_size,img_size))
        mouth_images.append(img)
        mouth_labels.append(1 if label=='yawn' else 0)

# Convert to numpy arrays and normalize
eye_images = np.array(eye_images)/255.0
eye_labels = np.array(eye_labels)

mouth_images = np.array(mouth_images)/255.0
mouth_labels = np.array(mouth_labels)

# Split datasets
X_train_eye, X_val_eye, y_train_eye, y_val_eye = train_test_split(eye_images, eye_labels, test_size=0.2, random_state=42)
X_train_mouth, X_val_mouth, y_train_mouth, y_val_mouth = train_test_split(mouth_images, mouth_labels, test_size=0.2, random_state=42)

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_cnn():
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=(img_size,img_size,3)))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Eye model
eye_model = build_cnn()
eye_model.summary()
eye_model.fit(X_train_eye, y_train_eye, validation_data=(X_val_eye, y_val_eye), epochs=10, batch_size=32)
eye_model.save('/content/eye_model.h5')

# Mouth model
mouth_model = build_cnn()
mouth_model.fit(X_train_mouth, y_train_mouth, validation_data=(X_val_mouth, y_val_mouth), epochs=10, batch_size=32)
mouth_model.save('/content/mouth_model.h5')


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 683,329 (2.61 MB)

 Trainable params: 683,329 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 11s 244ms/step - accuracy: 0.7228 - loss: 0.5423 - val_accuracy: 0.9347 - val_loss: 0.1991
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 9s 234ms/step - accuracy: 0.9419 - loss: 0.1870 - val_accuracy: 0.9519 - val_loss: 0.1352
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 11s 248ms/step - accuracy: 0.9674 - loss: 0.1183 - val_accuracy: 0.9519 - val_loss: 0.1037
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 9s 244ms/step - accuracy: 0.9694 - loss: 0.0890 - val_accuracy: 0.9519 - val_loss: 0.0955
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 9s 209ms/step - accuracy: 0.9700 - loss: 0.0793 - val_accuracy: 0.9794 - val_loss: 0.0541
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 10s 209ms/step - accuracy: 0.9745 - loss: 0.0559 - val_accuracy: 0.9656 - val_loss: 0.0765
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 11s 219ms/step - accuracy: 0.9821 - loss: 0.0468 - val_accuracy: 0.9897 - val_loss: 0.0322
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 11s 250ms/step - accuracy: 0.9813 - loss: 0.0385 - val_accurac

Epoch 1/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 11s 217ms/step - accuracy: 0.5324 - loss: 0.6890 - val_accuracy: 0.6414 - val_loss: 0.6065
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 9s 248ms/step - accuracy: 0.6119 - loss: 0.5969 - val_accuracy: 0.6862 - val_loss: 0.5758
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 10s 251ms/step - accuracy: 0.6787 - loss: 0.5691 - val_accuracy: 0.7310 - val_loss: 0.5393
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 8s 207ms/step - accuracy: 0.7130 - loss: 0.5363 - val_accuracy: 0.7172 - val_loss: 0.5378
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 9s 246ms/step - accuracy: 0.7081 - loss: 0.5164 - val_accuracy: 0.7241 - val_loss: 0.5107
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 9s 244ms/step - accuracy: 0.7580 - loss: 0.4820 - val_accuracy: 0.8000 - val_loss: 0.4039
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 8s 206ms/step - accuracy: 0.8112 - loss: 0.4027 - val_accuracy: 0.8931 - val_loss: 0.2856
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 9s 243ms/step - accuracy: 0.9036 - loss: 0.2586 - val_accuracy: 

In [28]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np

eye_model = load_model('/content/eye_model.h5')
mouth_model = load_model('/content/mouth_model.h5')

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        face = frame[y:y+h, x:x+w]

        # Eyes
        eyes = eye_cascade.detectMultiScale(face)
        for (ex,ey,ew,eh) in eyes:
            eye_img = face[ey:ey+eh, ex:ex+ew]
            eye_img = cv2.resize(eye_img, (img_size,img_size))/255.0
            eye_img = np.expand_dims(eye_img, axis=0)
            pred_eye = eye_model.predict(eye_img)
            label_eye = "Closed" if pred_eye < 0.5 else "Open"
            cv2.rectangle(frame, (x+ex,y+ey), (x+ex+ew, y+ey+eh), (255,0,0), 2)
            cv2.putText(frame, label_eye, (x+ex, y+ey-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0),2)

        # Mouth (lower half of face)
        mouth_img = face[h//2:h, :]
        mouth_img = cv2.resize(mouth_img, (img_size,img_size))/255.0
        mouth_img = np.expand_dims(mouth_img, axis=0)
        pred_mouth = mouth_model.predict(mouth_img)
        label_mouth = "Yawn" if pred_mouth > 0.5 else "No Yawn"
        cv2.putText(frame, label_mouth, (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255),2)

    cv2.imshow('Drowsiness Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [29]:
# Testing
from tensorflow.keras.models import load_model
import cv2
import numpy as np

eye_model = load_model('/content/eye_model.h5')
mouth_model = load_model('/content/mouth_model.h5')


In [38]:
img_size = 64

# Example test image path
test_eye_img_path = '/content/drowsiness_dataset/train/Closed/_0.jpg'
eye_img = cv2.imread(test_eye_img_path)
eye_img = cv2.cvtColor(eye_img, cv2.COLOR_BGR2RGB)
eye_img = cv2.resize(eye_img, (img_size,img_size))
eye_img = eye_img / 255.0
eye_img = np.expand_dims(eye_img, axis=0)  # batch dimension


**Eye Testing**

In [40]:
pred_eye = eye_model.predict(eye_img)
label_eye = "Open" if pred_eye < 0.5 else "Closed"
print("Eye State:", label_eye)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Eye State: Closed


**Mouth Testing**

In [42]:
test_mouth_img_path = '/content/drowsiness_dataset/train/yawn/1.jpg'
mouth_img = cv2.imread(test_mouth_img_path)
mouth_img = cv2.cvtColor(mouth_img, cv2.COLOR_BGR2RGB)
mouth_img = cv2.resize(mouth_img, (img_size,img_size))
mouth_img = mouth_img / 255.0
mouth_img = np.expand_dims(mouth_img, axis=0)

pred_mouth = mouth_model.predict(mouth_img)
label_mouth = "Yawn" if pred_mouth > 0.5 else "No Yawn"
print("Mouth State:", label_mouth)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Mouth State: Yawn


**Accuracy**

In [43]:
val_loss, val_acc = eye_model.evaluate(X_val_eye, y_val_eye)
print("Eye Model Accuracy:", val_acc)

val_loss, val_acc = mouth_model.evaluate(X_val_mouth, y_val_mouth)
print("Mouth Model Accuracy:", val_acc)


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.9833 - loss: 0.0731
Eye Model Accuracy: 0.9896907210350037
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.9392 - loss: 0.1568
Mouth Model Accuracy: 0.9586206674575806
